In [3]:
import pandas as pd
import os
import numpy as np
import openpyxl

In [2]:
excel_file = 'process map.xlsx'
rows = 585

In [3]:
cols_dict = {
    'F_m': 'failure_mode',
    'Subprocess': 'subprocess_id',
    'Step': 'step_id',
    'Found also in other steps such as': 'found_in_other_steps',
    'Potential Effect': 'potential_effect',
    'Potentail Causes': 'potentail_causes',
    'Severity': 'severity',
    'RPN': 'rpn',
    'Mitigation Strategy / detaction mode': 'mitigation_strategy',
    'Process_type': 'process_type',
    'Software': 'software',
    'ABR': 'abr'
}

In [4]:
path = os.path.join('data', excel_file)
df_excel = pd.DataFrame(pd.read_excel(path, nrows=rows, usecols=list(cols_dict.keys())))
df_excel.rename(columns=cols_dict, inplace=True)

### subporcess_id

In [5]:
df_excel['subprocess_id'] = df_excel['subprocess_id'].str.strip().str.split('.').str[0]
df_excel.loc[df_excel['subprocess_id'].str.contains('A', na=False), 'subprocess_id'] = -1 # A.
df_excel['subprocess_id'] = df_excel['subprocess_id'].fillna(0) # unknown 

### step_id 

In [6]:
df_excel['step_id'] = df_excel['step_id'].str.strip().str.split(' ').str[0].str.split('.').str[1]
df_excel.loc[df_excel['step_id'].str.contains('A', na=False), 'step_id'] = -1 # A
df_excel.loc[df_excel['step_id'].str.contains('\?', na=False), 'step_id'] = 0 # unknown
df_excel['step_id'] = df_excel['step_id'].fillna(0) # unknown 

### process_id

In [7]:
df_excel['process_id'] = df_excel.\
apply(lambda r: (r['subprocess_id'] + '-' + r['step_id']) if r['subprocess_id'] != -1 and r['subprocess_id'] != 0 and r['step_id'] != 0 and r['step_id'] != -1 else np.NaN, axis=1)

### severity

In [8]:
four_scale_dict = {
    1: 1.0, # (0, 1]
    2: 2.0, # (1, 2]
    3: 3.0, # (2, 3]
    4: 4.0 # (3, 4]
}

five_scale_dict = {
    2: 1.0, # [1, 2)
    3: 2.0, # [2, 3)
    4: 3.0, # [3, 4)
    6: 4.0 # [4, 6)
}

ten_scale_dict = {
    4: 1.0, # [1, 4) 
    7: 2.0, # [4, 7)
    9: 3.0, # [7, 9)
    11: 4.0 # [9, 10]
}

In [9]:
df_excel[['severity', 'scale']] = df_excel['severity'].str.strip().str.split(' ', 1, expand=True)

df_excel['severity'] = pd.to_numeric(df_excel['severity'], errors='coerce')
df_excel['severity'] = df_excel['severity'].fillna(0) # unknown 

for index, row in df_excel.iterrows():
    if row['scale'] is None or '10' in str(row['scale']):
        for key, value in ten_scale_dict.items():
            if row['severity'] < key:
                df_excel.at[index, 'severity'] = value   
                break
    elif '4' in str(row['scale']):
        for key, value in four_scale_dict.items():
            if row['severity'] <= key:
                df_excel.at[index, 'severity'] = value 
                break
    elif '5' in str(row['scale']):
        for key, value in five_scale_dict.items():
            if row['severity'] < key:
                df_excel.at[index, 'severity'] = value  
                break
                
df_excel = df_excel.drop('scale', axis=1)

/var/folders/2s/lz9qf7xj08lfknpkfv44qn580000gn/T/ipykernel_95587/1464670255.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_excel[['severity', 'scale']] = df_excel['severity'].str.strip().str.split(' ', 1, expand=True)


### RPN

In [12]:
df_excel['rpn'] = df_excel['rpn'].str.strip().str.split('(').str[0].str.replace(',', '.')

In [13]:
df_excel.to_excel("test.xlsx", index=False)  